---
layout: post
title: Terraform Automation Plan
description: A plan to assist in creating AWS workspace creation using Terraform.  The purpose of this automation is to provide Student Desktops in the cloud.
author: Aaron Anand, Aadit Gupta
courses: { csp: {week: 20, categories: [6.B]}, csa: {week: 20} }
categories: [C7.0, C7.1, C7.2]
type: devops
---

## Automation Planning
`Cloud Computing`.  Traditionally, classroom have been service by on premise  desktops; costs can be high and machine aging is rapid. The purpose of this project is to build workspaces in the cloud. The students would use their District issued Chromebook to access these workspaces which could enable them in their Computer Science Pathway.   The numbers indicated that this could be much more affordable and aging/refresh cycles go away.  

`Automating Cloud Setups`.  The following illustration assumes a `Terraform` recipe being pushed to each `AWS EC2`, the electric cloud computer is generated into a `Ubuntu Linux` workspace.   The names `kasm1.ncs.com, kasm2.ncs.com, ...` represent machines using `KASM workspaces`, a containerized desktop software provider.  Each workspace contains a `desktop environment` and `software tools` used in the Computer Science classroom.  


Illustration of building KASM workspaces (kasm1 to kasmN)
```
            AWS EC2 Ubuntu Linux instances scaled (1 to N)
            |-> kasm1.ncs.com - 5 student logins / 3 active sessions
            |
            |-> kasm2.ncs.com - 5 student logins / 3 active sessions
Terraform-->|
 (recipe)   |-> kasm3.ncs.com - 5 student logins / 3 active sessions
            |
            |-> kasm4.ncs.com - 5 student logins / 3 active sessions
            |
            |-> kasmN.ncs.com - 5 student logins / 3 active sessions

```
`Recipe`.  The term `recipe` is used to describe all the elements needed to build the AWS EC2 Ubunto machines and KASM workspaces.  Terraform is the tool that drives the process.  A Terraform recipe, as in a cooking recipe, has many steps, intermediate processes, and outcomes.  These step, processes and outcomes are quite involved and compose most of the content in this document.

`Scaling and Management`. To scale number of students using the workspaces, each machine can have up to `5 user logins` per instance, but according to resources would only adequetly serve `3 active sessions`.  There are many additional Terraform, AWS and/or KASM features to allocate resources, scale resources, and timeout inactive workspace sessions.  Additionally, `machines can be put to sleep`, for instance 11pm to 7am daily to save on cost;`woken up` based off of demand weekends and holidays.  In addition to the recipe to put the system in place, there are many additional process to manage usage and control costs.


### Setup Requirements (Part 1 - System)
`System`. These are features that get the KASM system running on AWS EC2 Ubuntu.    The specification have been captured by proof of concept analysis.

#### AWS EC2 Instances
Configuration
- Region Oregon (us-west-2)
- Instance t2.medium (2 x CPU, 4 Gib RAM)
- Architecture x86
- Use “kasm” key pair
- Storage 60 Gib gp3 general purpose SD
- Allow SSH, HTTP, HTTPS

#### KASM single server install 1.13.1 
scripting commands

```bash
# Reference: https://kasmweb.com/docs/latest/install/single_server_install.html
cd /tmp
curl -O https://kasm-static-content.s3.amazonaws.com/kasm_release_1.13.1.421524.tar.gz
tar -xf kasm_release_1.13.1.421524.tar.gz

# Automate prompts for EULA, Swap Partition,and Port
sudo bash kasm_release/install.sh --accept-eula --swap-size 8192 -L 8443
```

### Setup Requirements (Part 2 - Networking)
`Networking`.  The purpose of this step is to provide friendly internet access names to the students.  Additionally, we hope to enable as much setup and verification of these steps in Terraform and AWS CLI as possible.

#### AWS EC2 elastic IPs, permanent ips
Each AWS EC2 Ubuntu KASM instance requires a Virtual Private Cloud VPC elastic IP address.  This will result in allocation of IP address that needs to be assigned to server.  
- Region: Oregon (us-west-2)
- Name: kasm1
- Allocation ID: i-0ef40a9caee208041 (kasm1 ec2 instance ID)

#### AWS Route 53, kasm DNS names to elastic IPs
AWS Route 53 requires setup DNS subdomain for each KASM server elastic ip address.  This example shows description of keys and values used in Route 53.
- Hosted Zone: nighthawkcodingsociety.com
- Record Name: kasm1.nighthawkcodingsociety.com	
- Type: A
- Routing Policy: Simple
- Alias: No
- Value/Route traffic to: 100.21.190.187
- TTL (seconds): 60

#### Nginx configuration to reverse proxy kasm DNS name to port
Nginx needs to be configured to route kasm1.nighthawkcodingsociety.com request to port 8443.  FYI, port was setup with -L option during KASM install.

Nginx needs to be install on EC2 instance.

```bash
sudo apt install nginx
```

Nginx configuration file, `kasm1` is created with content as shown, only difference in configuration per running host (kasm1, kasm2, ...) is the server_name subdomain.

```json
server {
    server_name kasm1.nighthawkcodingsociety.com;
    listen 80;

    location / {
         # The following configurations must be configured when proxying to Kasm Workspaces

         # WebSocket Support
         proxy_set_header        Upgrade $http_upgrade;
         proxy_set_header        Connection "upgrade";

         # Host and X headers
         proxy_set_header        Host $host;
         proxy_set_header        X-Real-IP $remote_addr;
         proxy_set_header        X-Forwarded-For $proxy_add_x_forwarded_for;
         proxy_set_header        X-Forwarded-Proto $scheme;

         # Connectivity Options
         proxy_http_version      1.1;
         proxy_read_timeout      1800s;
         proxy_send_timeout      1800s;
         proxy_connect_timeout   1800s;
         proxy_buffering         off;

         # Allow large requests to support file uploads to sessions
         client_max_body_size 10M;

         # Endpoint of service
         proxy_pass https://localhost:8443;
    }
}
```

Nginx configuration file is placed in /etc/nginx/sites-available and then is symbolically linked to /etc/nginx/sites-enabled.
- nginx filename: `kasm1`
- file path: `/etc/nginx/sites-available/`

Activate kasm1 reverse proxy DNS request to Application Endpoint

```bash
rm -f /etc/nginx/sites-enabled/kasm1
ln -s /etc/nginx/sites-available/kasm1 /etc/nginx/sites-enabled
sudo systemctl restart nginx
```


#### Certbot certificates for DNS name to HTTPs
Certbot providesfree certificates to application endpoints that serve their content over HTTPS

Install Certbot
```bash
sudo snap install core; sudo snap refresh core
sudo snap install --classic certbot
sudo ln -s /snap/bin/certbot /usr/bin/certbot
```

Assign certificate to nginx hosts.

```bash
sudo certbot --nginx --non-interactive --agree-tos -m kasm1.nighthawkcodingsociety.com
```

### Setup Requirements (Part 3 - Workspaces)
`Workspace`.  The purpose of this step is to configure KASM workspaces to meet the needs of CSSE, CSP, and CSA classes.  This includes all development tools. 

As KASM workspace is built by docker, common developer tools like `make` were found missing.   KASM workspaces do not support `sudo apt` at runtimes, so tools need to be comprehensive.

Docker Workspace and Registry must match version requirements of system setup (1.13.1)
- Docker workspace Video https://www.youtube.com/watch?v=BGP69_f1wq0
- Registry video https://www.youtube.com/watch?v=Y6ggn9McIwI

#### Tools Requirements
This shell script is used to verfy and configure 

In [ ]:
#!/bin/bash

# Function to check if a line exists in the .bashrc file
line_exists_in_bashrc() {
  grep -Fxq “$1" ~/.bashrc
}
# Function to add lines to the .bashrc file if they don’t exist
add_to_bashrc() {
  if ! line_exists_in_bashrc “$1”; then
    echo “$1" >> ~/.bashrc
  fi
}
# Function to install Visual Studio Code
install_vscode() {
  if ! command -v code &>/dev/null; then
    echo “Installing Visual Studio Code...”
    # Add VSCode repository and key
    wget -qO- https://packages.microsoft.com/keys/microsoft.asc | gpg --dearmor > packages.microsoft.gpg
    sudo mv packages.microsoft.gpg /etc/apt/trusted.gpg.d/microsoft.gpg
    sudo sh -c ‘echo “deb [arch=amd64] https://packages.microsoft.com/repos/vscode stable main” > /etc/apt/sources.list.d/vscode.list’
    # Update package index and install VSCode
    sudo apt update
    sudo apt install -y code
    echo “Visual Studio Code installed successfully.”
  else
    echo “Visual Studio Code is already installed.”
  fi
}
# Function to install ruby
install_ruby() {
# Check if ruby command is available
  if ! command -v ruby &>/dev/null; then
    echo “Installing Ruby...”
    sudo apt update
	sudo apt-get install ruby-full build-essential zlib1g-dev
  else
    echo “Ruby is already installed.”
  fi
}
# Function to install MiniConda and Build environment
install_miniconda() {
  if ! command -v conda &>/dev/null; then
    echo “Installing MiniConda...”
    # Download MiniConda installer
    wget -q https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -O ~/miniconda.sh
    # Install MiniConda
    bash ~/miniconda.sh -b -p $HOME/miniconda
    rm ~/miniconda.sh
  else
    echo “MiniConda is already installed.”
  fi
}
# The environment_activate function effectively sets up the required environment configurations by adding lines to the .bashrc file only if they don’t already exist. This ensures that the modifications to the .bashrc file are performed only once and avoids duplication.
environment_activate() {
  add_to_bashrc `# local environment requirement`
  add_to_bashrc ‘export PATH=“$HOME/miniconda/bin:$PATH”’
  add_to_bashrc ‘export GEM_HOME=“$HOME/gems”’
  add_to_bashrc ‘export PATH=“$HOME/gems/bin:$PATH”’
  conda init bash
  echo “Tools and environment setup. Please restart your terminal or run ‘source ~/.bashrc’ to activate it.”
}
# Main script
install_vscode
install_ruby
install_miniconda
environment_activate

This Docker file is in KASM format with required head and tail sections, all customization must be done in between

In [ ]:

FROM kasmweb/core-ubuntu-focal:1.13.1
USER root
ENV HOME /home/kasm-default-profile
ENV STARTUPDIR /dockerstartup
ENV INST_SCRIPTS $STARTUPDIR/install
ENV CONDA_DIR /opt/conda
ENV PATH $CONDA_DIR/bin:$PATH
WORKDIR $HOME

######### Customize Container Here ###########
# Install Google Chrome
COPY ./src/ubuntu/install/chrome $INST_SCRIPTS/chrome/
RUN bash $INST_SCRIPTS/chrome/install_chrome.sh  && rm -rf $INST_SCRIPTS/chrome/
# Install VSCode
COPY ./src/ubuntu/install/vs_code $INST_SCRIPTS/vs_code/
RUN bash $INST_SCRIPTS/vs_code/install_vs_code.sh  && rm -rf $INST_SCRIPTS/vs_code/
# Install Anaconda
RUN cd /tmp/ && wget https://repo.anaconda.com/archive/Anaconda3-2023.07-1-Linux-x86_64.sh \
    && bash Anaconda3-20*-Linux-x86_64.sh -b -p /opt/anaconda3 \
    && rm -r /tmp/Anaconda3-20*-Linux-x86_64.sh \
    && echo ‘source /opt/anaconda3/bin/activate’ >> /etc/bash.bashrc \
    && bash -c “source /opt/anaconda3/bin/activate \
        && conda update -n root conda  \
        && conda update --all \
        && conda clean --all” \
    && /opt/anaconda3/bin/conda config --set ssl_verify /etc/ssl/certs/ca-certificates.crt \
    && /opt/anaconda3/bin/conda install pip \
    && mkdir -p /home/kasm-user/.pip \
    && chown -R 1000:1000 /opt/anaconda3 /home/kasm-default-profile/.conda/ && conda install -y jupyter
######### End Customizations ###########

RUN chown 1000:0 $HOME
RUN $STARTUPDIR/set_user_permission.sh $HOME
ENV HOME /home/kasm-user
WORKDIR $HOME
RUN mkdir -p $HOME && chown -R 1000:0 $HOME
USER 1000 (edi

## Automation Tool Setup
Tool installation and verification.  This is a preparation step prior to using Terraform. Below are commands to configure Amazon Web Service Command Line Interface `awscli`, verify configuration `sts`, and test the Simple Storage Service `s3` and `s3api`.
- `Terraform` is a tool that lets you build, change, and version cloud resources safely and efficiently.
- `AWS CLI` (Amazon Web Service Command Line Interface) is a tool to manage your AWS services.

The step below are simple instruction to install, setup, and verify these automation tools on your desktop.

In [ ]:
# 
# Automation key tools
$ brew install terraform
$ brew install awscli  

# Credentials setup (~/.aws/credentials)
$ aws configure  # prompts with keys. user adds values values
# Find User Access Keys in AWS IAM, answer as follows 
<<dialog #begin
AWS Access Key ID [****************PQML]:
AWS Secret Access Key [****************Smcy]:
Default region name [us-west-2]:
Default output format [json]:
dialog #end

# Credentials verification with Security Token Service (sts)
$ aws sts get-caller-identity
# Notice return of IAM user
<<stsoutput #begin

  UserId: ****************2B6D7
  Account: ******879144,
  Arn: arn:aws:iam::******879144:user/j***21

stsoutput #ends

# Verify access to view a service, using s3 - 
$ aws s3 ls
# This is a list of s3 buckets, notice terra-auto directory
<<s3output #begin
2022-09-28 13:38:43 aws-cloudtrail-logs-542024879144-84383535
2022-08-09 07:00:27 elasticbeanstalk-us-west-1-542024879144
2023-05-25 12:47:32 loginzeen
2022-05-19 01:52:48 prehistoric-pals
2023-07-31 14:14:25 terra-auto
s3output #ends

# Verify and play with commands to write to a service, using s3api
$ aws s3api list-objects --bucket terra-auto
$ aws s3api put-object --bucket terra-auto --key myfile.py
$ aws s3api list-objects --bucket terra-auto